In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Define the directory containing the images
image_directory = './'  # Assuming images are in the current directory

# Initialize a list to store image paths
image_files = []

# List all files in the directory and filter for image files
for file_name in os.listdir(image_directory):
    if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
        image_files.append(os.path.join(image_directory, file_name))

# Function to read and display images using OpenCV
def read_and_display_images(image_paths):
    for image_path in image_paths:
        # Read the image
        image = cv2.imread(image_path)
        if image is not None:
            # Display the image
            cv2.imshow('Image', image)
            cv2.waitKey(1000)  # Display each image for 1 second
        else:
            print(f"Failed to read image: {image_path}")

    # Close all OpenCV windows
    cv2.destroyAllWindows()


In [ ]:
# Call the function with the list of image files
read_and_display_images(image_files)

In [ ]:
image_files

In [ ]:
def show_img(image,title='',fig_sz=(16,12)):
    plt.figure(figsize=fig_sz)
    plt.imshow(image, cmap='gray' if len(image.shape) == 2 else None)
    plt.title(title)
    plt.axis('off')
    plt.show()

In [ ]:
def show_channel(channel, title='', fig_sz = (16,12)):
    plt.figure(figsize=fig_sz)
    plt.imshow(channel, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

In [ ]:
# Read the first image in the list
image = cv2.imread(image_files[9])
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
show_img(image_rgb,"milky way og")

In [ ]:
# Apply median blur to each channel
ks = 5
b_channel = cv2.medianBlur(image[:, :, 0], ks)
g_channel = cv2.medianBlur(image[:, :, 1], ks)
r_channel = cv2.medianBlur(image[:, :, 2], ks)

# Merge the channels back
image_denoised = cv2.merge([r_channel, g_channel, b_channel])

# Display the denoised color image
show_img(image_denoised, 'Denoised Color Image')

In [ ]:
def unsharp_masking(image,ks=9,sd=10,wt_original=1.5,wt_blurred=-0.5,scalar=0):
    blurred = cv2.GaussianBlur(image, (ks, ks), sd)
    sharpened = cv2.addWeighted(image, wt_original, blurred, wt_blurred, scalar)
    r_channel, g_channel, b_channel = cv2.split(sharpened)
    combined_image1 = cv2.merge((b_channel, g_channel, r_channel))
    combined_image2 = cv2.merge((r_channel, g_channel, b_channel))
    
    show_img(combined_image1)
    show_img(combined_image2)
    
    return [combined_image1,combined_image2]

In [ ]:
combined_image = unsharp_masking(image)

In [ ]:
img2 = cv2.imread(image_files[13])
img21 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
show_img(img21)

img22 = unsharp_masking(img21)

In [ ]:
def proc_stars(proc_img_gray,threshold_value,show_imgs = False, figsz = (4,3)):
    
    _, thresholded_img = cv2.threshold(proc_img_gray, threshold_value, 255, cv2.THRESH_BINARY)
    
    kernel = np.ones((3, 3), np.uint8)
    cleaned_image = cv2.morphologyEx(thresholded_img, cv2.MORPH_OPEN, kernel, iterations=2)
    cleaned_image = cv2.morphologyEx(cleaned_image, cv2.MORPH_CLOSE, kernel, iterations=2)
    
    # Find contours of the stars
    contours, _ = cv2.findContours(cleaned_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw contours on the original binary image for visualization
    contour_image = cv2.cvtColor(cleaned_image, cv2.COLOR_GRAY2BGR)
    cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 1)

    # Count the number of stars
    num_stars = len(contours)
    print(f"Threshold: {threshold_value}, Number of stars detected: {num_stars}")
    
    if show_imgs:
        # Display the main, cleaned and contour image
        show_img(thresholded_img,fig_sz=figsz)
        show_img(cleaned_image, f'Cleaned Binary Image at threshold {threshold_value}', fig_sz=figsz)
        show_img(contour_image, f'Star Contours at threshold {threshold_value}', fig_sz=figsz)

In [ ]:
proc_img = img22[1]

# Convert to grayscale if not already
if len(proc_img.shape) == 3:
    proc_img_gray = cv2.cvtColor(proc_img, cv2.COLOR_RGB2GRAY)
else:
    proc_img_gray = proc_img

# Apply intensity thresholding
threshold_values = [(i*16)-1 for i in range(4,9)]
for threshold_value in threshold_values:
    if threshold_value == min(threshold_values):
        proc_stars(proc_img_gray,threshold_value,show_imgs=True,figsz=(8,6))
    elif threshold_value == max(threshold_values):
        proc_stars(proc_img_gray,threshold_value,show_imgs=True)
    else:
        proc_stars(proc_img_gray,threshold_value)

In [ ]:
x = [(i*16)-1 for i in range(1,17)]
x

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchviz import make_dot

# Define the neural network
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(3, 5)  # 3 input features to 5 hidden nodes
        self.dropout = nn.Dropout(0.5)  # Dropout with 50% probability
        self.fc2 = nn.Linear(5, 2)  # 5 hidden nodes to 2 output nodes

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # Apply ReLU activation function
        x = self.dropout(x)  # Apply dropout
        x = self.fc2(x)  # Output layer
        return x

# Create the network
net = SimpleNN()

# Example input
input_data = torch.randn(1, 3)  # Batch size of 1, 3 input features

# Forward pass
output_data = net(input_data)

# Visualize the network
make_dot(output_data, params=dict(net.named_parameters())).render("network", format="png")


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchviz import make_dot

# Define the neural network
class FashionMNISTNN(nn.Module):
    def __init__(self):
        super(FashionMNISTNN, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)  # Input layer (28x28 pixels) to 128 hidden nodes
        self.fc2 = nn.Linear(128, 64)  # Hidden layer to 64 hidden nodes
        self.fc3 = nn.Linear(64, 10)  # Hidden layer to 10 output nodes (10 classes)
        self.dropout = nn.Dropout(0.5)  # Dropout with 50% probability

    def forward(self, x):
        x = x.view(-1, 28*28)  # Flatten the input image
        x = torch.relu(self.fc1(x))  # ReLU activation
        x = self.dropout(x)  # Apply dropout
        x = torch.relu(self.fc2(x))  # ReLU activation
        x = self.fc3(x)  # Output layer
        return x

# Initialize the network
net = FashionMNISTNN()

# Define a loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

# Load FashionMNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.FashionMNIST(root='./data/FashionMNIST', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

# Example input from the dataset
dataiter = iter(trainloader)
images, labels = next(dataiter)  # Use the next function to get a batch
images = images[:1]  # Take a single example for visualization

# Forward pass to visualize the network
output = net(images)

# Visualize the network
make_dot(output, params=dict(net.named_parameters())).render("fashion_mnist_nn", format="png")

# Training loop
for epoch in range(2):  # Loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()  # Zero the parameter gradients

        outputs = net(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Optimize

        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

# Save the trained model
torch.save(net.state_dict(), 'fashion_mnist_nn.pth')


In [ ]:
!pip install pydot

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

# Load the FashionMNIST dataset
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize the data
x_train, x_test = x_train / 255.0, x_test / 255.0

# Define the neural network model
model = models.Sequential([
    layers.Flatten(input_shape=(28, 28)),  # Flatten the input image
    layers.Dense(128, activation='relu'),  # Hidden layer with ReLU activation
    layers.Dropout(0.5),  # Dropout layer to prevent overfitting
    layers.Dense(64, activation='relu'),  # Another hidden layer with ReLU activation
    layers.Dense(10)  # Output layer with 10 nodes (one for each class)
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# Save the model
model.save('fashion_mnist_model.h5')


In [ ]:
import os
logdir = os.path.join("logs", "fashion_mnist")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

# Train the model with TensorBoard callback
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test), callbacks=[tensorboard_callback])


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import os

# Load the FashionMNIST dataset
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize the data
x_train, x_test = x_train / 255.0, x_test / 255.0

# Define the neural network model
model = models.Sequential([
    layers.Flatten(input_shape=(28, 28)),  # Flatten the input image
    layers.Dense(128, activation='relu'),  # Hidden layer with ReLU activation
    layers.Dropout(0.5),  # Dropout layer to prevent overfitting
    layers.Dense(64, activation='relu'),  # Another hidden layer with ReLU activation
    layers.Dense(10)  # Output layer with 10 nodes (one for each class)
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Set up TensorBoard callback
logdir = os.path.join("logs", "fashion_mnist")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

# Train the model with TensorBoard callback
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test), callbacks=[tensorboard_callback])

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# Save the model
model.save('fashion_mnist_model.h5')

# Save the model architecture as an image
tf.keras.utils.plot_model(model, to_file='fashion_mnist_model.png', show_shapes=True, show_layer_names=True)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import os

# hyperparameters:


# Load the FashionMNIST dataset
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize the data
x_train, x_test = x_train / 255.0, x_test / 255.0

# Define the neural network model
model = models.Sequential([
    layers.Flatten(input_shape=(28, 28)),  # Flatten the input image
    layers.Dense(128, activation='relu'),  # Hidden layer with ReLU activation
    layers.Dropout(0.5),  # Dropout layer to prevent overfitting
    layers.Dense(64, activation='relu'),  # Another hidden layer with ReLU activation
    layers.Dense(10)  # Output layer with 10 nodes (one for each class)
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Set up TensorBoard callback
logdir = os.path.join("logs", "fashion_mnist")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

# Train the model with TensorBoard callback
model.fit(x_train, y_train, epochs=25, validation_data=(x_test, y_test), callbacks=[tensorboard_callback])

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# Save the model
model.save('fashion_mnist_model.h5')

# Save the model architecture as an image
tf.keras.utils.plot_model(model, to_file='fashion_mnist_model.png', show_shapes=True, show_layer_names=True)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import os

# Load the FashionMNIST dataset
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize the data
x_train, x_test = x_train / 255.0, x_test / 255.0

# Reshape the data to include the channel dimension
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# Define the Convolutional Neural Network model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),  # First convolutional layer
    layers.MaxPooling2D((2, 2)),  # First max pooling layer
    layers.Conv2D(64, (3, 3), activation='relu'),  # Second convolutional layer
    layers.MaxPooling2D((2, 2)),  # Second max pooling layer
    layers.Conv2D(64, (3, 3), activation='relu'),  # Third convolutional layer
    layers.Flatten(),  # Flatten the 3D output to 1D
    layers.Dense(128, activation='relu'),  # Fully connected layer with ReLU activation
    layers.Dropout(0.5),  # Dropout layer to prevent overfitting
    layers.Dense(10)  # Output layer with 10 nodes (one for each class)
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Set up TensorBoard callback
logdir = os.path.join("logs", "fashion_mnist")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

# Train the model with TensorBoard callback
model.fit(x_train, y_train, epochs=15, validation_data=(x_test, y_test), callbacks=[tensorboard_callback])

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# Save the model
model.save('fashion_mnist_cnn.h5')

# Save the model architecture as an image
tf.keras.utils.plot_model(model, to_file='fashion_mnist_cnn.png', show_shapes=True, show_layer_names=True)


In [2]:
import tensorflow as tf

# Check TensorFlow version
print("TensorFlow version:", tf.__version__)

# Check if TensorFlow was built with CUDA (GPU) support
print("Built with CUDA support:", tf.test.is_built_with_cuda())
print("Built with GPU support:", tf.test.is_built_with_gpu_support())

# List available physical devices
print("Available physical devices:")
physical_devices = tf.config.list_physical_devices()
for device in physical_devices:
    print(device)

# Check for GPU availability
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPUs are available.")
    for gpu in gpus:
        print(gpu)
else:
    print("GPUs are not available.")

# Check detailed device information
from tensorflow.python.client import device_lib
print("Detailed device information:")
print(device_lib.list_local_devices())

# Check if TensorFlow is using the GPU
if tf.config.list_physical_devices('GPU'):
    print("TensorFlow is using the GPU.")
else:
    print("TensorFlow is using the CPU.")


TensorFlow version: 2.17.0
Built with CUDA support: False
Built with GPU support: False
Available physical devices:
PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
GPUs are not available.
Detailed device information:
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3636850126378938361
xla_global_id: -1
]
TensorFlow is using the CPU.


In [ ]:
import pandas as pd
eg = egs.split('\n')
data = [line.split('\t') for line in eg]
df = pd.DataFrame(data[1:], columns=data[0])
print(len(df))
print(df.to_string())

In [ ]:
eg2 = egs2.split('\n')
data2 = [line.split('\t') for line in eg]
df2 = pd.DataFrame(data2[1:], columns=data2[0])
df_combined = pd.concat([df, df2]).drop_duplicates().reset_index(drop=True)
print(len(df_combined))
print(df_combined.to_string())
df_combined.to_csv("examples")

In [ ]:
import gc

del egs, eg, data, df, egs2, eg2, data2, df2
gc.collect()

In [ ]:
from graphviz import Digraph

dot = Digraph()

# Define nodes
dot.node('A', 'Node A')
dot.node('B', 'Node B')
dot.node('C', 'Node C')
dot.node('D', 'Node D')

# Define edges
dot.edges(['AB', 'AC', 'BD', 'CD'])

# Save and render the graph
dot.render('graph', format='png')


In [ ]:
content = contents.split('\n')
for c in content:
    b = c.replace("    ", "\t")
    print(b)

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import matplotlib.pyplot as plt

def calculate_term_frequency(text):
    words = word_tokenize(text.lower())
    term_frequency = Counter(words)
    
    return term_frequency

tf = calculate_term_frequency(text)
sorted_tf = sorted(tf.items(), key=lambda x: (-x[1], x[0]))

for term, frequency in sorted_tf:
    print(f"{term}: {frequency}")

# Create histogram data
frequency_count = Counter(tf.values())

# Sort the frequency count by number of occurrences
sorted_frequency_count = sorted(frequency_count.items())

# Plot histogram
plt.figure(figsize=(21,9))
bars = plt.bar([str(item[0]) for item in sorted_frequency_count], [item[1] for item in sorted_frequency_count])
plt.bar([str(item[0]) for item in sorted_frequency_count], [item[1] for item in sorted_frequency_count])
plt.xlabel('Number of Occurrences')
plt.ylabel('Number of Words')
plt.title('Word Frequency Histogram')
# Add labels on top of each bar
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, int(yval), va='bottom') # va='bottom' to position the text below the bar edge
plt.grid()
plt.show()

In [ ]:

import math as m
import numpy as np

nnc = nn_contents.split('\n')
s_cnt = np.array([int(i.split()[-1]) for i in nnc])

# print(s_cnt)
print(f"total #slides = {np.sum(s_cnt)}\
        \ntotal #lectures = {len(s_cnt)}\
        \navg #slides per lec = {int(np.ceil(np.mean(s_cnt)))}\
        \nmin #slides = {np.min(s_cnt)}\
        \nmax #slides = {np.max(s_cnt)}")

In [ ]:
import os

# Specify the directory
dls = r"C:/Users/Siva/Downloads/Dua Lipa"

# List all files in the directory
for filename in os.listdir(dls):
    print(f"Processing file: {filename}")
    
    # Ensure the filename is long enough to contain a number and dot
    if len(filename) > 3 and filename[:2].isdigit() and filename[2] == '.':
        # Remove the number, dot, and any leading whitespace
        new_name = filename[4:].strip()
        
        # Construct full file paths
        old_file = os.path.join(dls, filename)
        new_file = os.path.join(dls, new_name)
        
        # Check if the new file name already exists
        if os.path.exists(new_file):
            print(f"Skipped: {new_name} already exists.")
        else:
            # Rename the file
            os.rename(old_file, new_file)
            print(f"Renamed: {filename} -> {new_name}")
    else:
        print(f"Skipped: {filename}")


In [ ]:
import os
import pandas as pd
from mutagen import File

# Specify the directory
directory = r"C:/Users/Siva/Downloads/Dua Lipa"

# List to hold details
file_details = []

# Iterate through the files in the directory
for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    
    # Check if it's a file (not a directory)
    if os.path.isfile(file_path):
        # Get file size in KB
        file_size = os.path.getsize(file_path) / (1024)
        
        # Get file duration using mutagen
        try:
            audio = File(file_path)
            duration = audio.info.length if audio and audio.info else None
        except Exception as e:
            duration = None
            print(f"Could not read duration for {filename}: {e}")
        
        # Add details to the list
        file_details.append({
            "File Name": filename,
            "File Size (KB)": round(file_size, 2),
            "Duration (s)": round(duration, 2) if duration else "Unknown",
            "Duration(mm:ss)": f"{int(duration // 60)}:{int(duration % 60):02d}" if duration else "Unknown"
        })

# Create a DataFrame
df = pd.DataFrame(file_details)

# Save the DataFrame to a CSV file
csv_path = os.path.join(directory, "file_details.csv")
df.to_csv(csv_path, index=False)

print(f"Details saved to {csv_path}")
